In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import re

from vietnamadminunits.parser.utils import key_normalize

import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path().resolve().parent.parent

In [3]:
# def create_sort(text, level=1):
#     if isinstance(text, str):
#         if level == 1:
#             text = re.sub(r'^Tỉnh\s|Thành phố\s', '', text, flags=re.IGNORECASE)
#         elif level == 2:
#             if re.search(r'^Quận\s\d{1,2}', text, flags=re.IGNORECASE):
#                 pass
#             else:
#                 text = re.sub(r'^Quận\s|Huyện\s|Thị xã\s|Thành phố\s', '', text, flags=re.IGNORECASE)
#         else:
#             if re.search(r'^Phường\s\d{1,2}', text, flags=re.IGNORECASE):
#                 pass
#             else:
#                 text = re.sub(r'^Phường\s|Thị trấn\s|Xã\s', '', text, flags=re.IGNORECASE)
#
#         return text.strip()
#     return text


def create_sort(text, level=1):
    if not isinstance(text, str):
        return text

    # Định nghĩa các tiền tố cần xóa theo cấp
    REMOVE_PREFIXES = {
        1: r'^(Tỉnh|Thành phố)\s',
        2: r'^(?!Quận\s\d{1,2})(Quận|Huyện|Thị xã|Thành phố)\s',
        3: r'^(?!Phường\s\d{1,2})(Phường|Thị trấn|Xã)\s',
    }

    pattern = REMOVE_PREFIXES.get(level)
    if pattern:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    return text.strip()


def create_keywords(row, level=1):
    district_type_acronym = {
        'Quận': 'q',
        'Thị xã': 'tx',
        'Thành phố': 'tp',
        'Huyện': 'h',
    }
    ward_type_acronym = {
        'Phường': 'p',
        'Thị trấn': 'tt',
        'Xã': 'x'
    }

    keywords = []
    if level == 1:
        keywords.append(row['provinceKey'])
        keywords.append(row['provinceShortKey'])
        if pd.notnull(row['provinceAlias']):
            aliases = json.loads(row['provinceAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    elif level == 2:
        keywords.append(row['districtKey'])

        if not row['districtShortKeyDuplicated']:
            keywords.append(row['districtShortKey'])
        else:
            keywords.append(key_normalize(f"{row['districtShortKey']} {row['districtType']}"))
            keywords.append(key_normalize(f"{district_type_acronym[row['districtType']]} {row['districtShortKey']}"))

        if row['districtShortDuplicated']:
            district_type = row['districtType']
            district_type_key = key_normalize(district_type)
            district_type_key_acronym = district_type_acronym[row['districtType']]
            acronym_keyword = re.sub(fr'^{district_type_key}', district_type_key_acronym, row['districtKey'])
            keywords.append(acronym_keyword)

        if pd.notnull(row['districtAlias']):
            aliases = json.loads(row['districtAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

        if re.search(r'^quan\d{1,2}', row['districtKey'], flags=re.IGNORECASE):
            keywords.append(row['districtKey'].replace('quan', 'q'))
            keywords.append(row['districtKey'].replace('quan', 'q.'))
            keywords.append(row['districtKey'].replace('quan', 'district'))

    else:
        if pd.notnull(row['wardKey']):
            keywords.append(row['wardKey'])


            if not row['wardShortKeyDuplicated']:
                keywords.append(row['wardShortKey'])
            else:
                keywords.append(key_normalize(f"{row['wardShortKey']} {row['wardType']}"))
                keywords.append(key_normalize(f"{ward_type_acronym[row['wardType']]} {row['wardShortKey']}"))


            if row['wardShortDuplicated']:
                ward_type = row['wardType']
                ward_type_key = key_normalize(ward_type)
                ward_type_key_acronym = ward_type_acronym[row['wardType']]
                acronym_keyword = re.sub(fr'^{ward_type_key}', ward_type_key_acronym, row['wardKey'])
                keywords.append(acronym_keyword)


            if pd.notnull(row['wardAlias']):
                aliases = json.loads(row['wardAlias'])
                for a in aliases:
                    keywords.append(key_normalize(a))

            if re.search(r'^phuong\d{1,2}', row['wardKey'], flags=re.IGNORECASE):
                keywords.append(row['wardKey'].replace('phuong', 'p'))
                keywords.append(row['wardKey'].replace('phuong', 'p.'))
                keywords.append(row['wardKey'].replace('phuong', 'ward'))
        else:
            return np.nan

    keywords = list(set(keywords))
    keywords = sorted(keywords, key=len, reverse=True)
    return json.dumps(keywords)

In [5]:
create_sort(text='Quận Tân Bình', level=2)

'Tân Bình'

In [3]:
df = pd.read_csv(BASE_DIR / 'data/processed/63_provinces_10040_wards_with_location.csv')

In [4]:
df

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,districtLon,districtBounds,districtGeoAddress,wardCode,wardLat,wardLon,wardBounds,wardGeoAddress,districtShortDuplicated,wardShortDuplicated
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Hà Nội,Ba Đình,Phúc Xá,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",1.0,21.046798,105.848112,"21.040199,105.842293 – 21.053464,105.855959","Phuc Xa, Ba Đình, Hanoi, Vietnam",False,False
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Hà Nội,Ba Đình,Trúc Bạch,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",4.0,21.044395,105.843174,"21.039976,105.836277 – 21.050836,105.850246","Truc Bach, Ba Đình, Hanoi, Vietnam",False,False
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Hà Nội,Ba Đình,Vĩnh Phúc,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",6.0,21.041731,105.809950,"21.035736,105.805872 – 21.047587,105.815001","Vĩnh Phúc, Ba Đình, Hà Nội, Vietnam",False,False
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Hà Nội,Ba Đình,Cống Vị,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",7.0,21.035280,105.809216,"21.031335,105.804777 – 21.037841,105.814554","Cống Vị, Ba Đình, Hà Nội, Vietnam",False,False
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Hà Nội,Ba Đình,Liễu Giai,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",8.0,21.037997,105.818756,"21.033027,105.812652 – 21.042991,105.821777","Liễu Giai, Ba Đình, Hà Nội, Vietnam",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10035,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Cà Mau,Ngọc Hiển,Viên An Đông,Huyện,Xã,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32239.0,8.637734,104.950567,"8.570243,104.878836 – 8.749535,104.996595","Viên An, Ngọc Hiển District, Ca Mau, Vietnam",False,False
10036,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Cà Mau,Ngọc Hiển,Viên An,Huyện,Xã,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32242.0,8.663728,104.833788,"8.567442,104.772921 – 8.737998,104.915485","Viên An, Ngọc Hiển District, Ca Mau, Vietnam",False,False
10037,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Cà Mau,Ngọc Hiển,Rạch Gốc,Huyện,Thị trấn,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32244.0,8.656311,105.020676,"8.596467,104.967112 – 8.679712,105.061312","Rạch Gốc, Ngọc Hiển District, Ca Mau, Vietnam",False,False
10038,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Cà Mau,Ngọc Hiển,Tân Ân,Huyện,Xã,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32245.0,8.653504,105.090815,"8.598292,105.007668 – 8.701729,105.142808","Tân Ân, Ngọc Hiển, Cà Mau, Vietnam",False,False


In [5]:
# ENRICH DATA
unit_cols = ['province', 'district', 'ward']
level_map = {
    'province': 1,
    'district': 2,
    'ward': 3
}

for col in unit_cols:
    # Create short version
    # level = level_map[col]
    # df[f"{col}Short"] = df[col].apply(create_sort, args=(level,))

    # Create key
    df[f"{col}Key"] = df[f"{col}"].apply(key_normalize)

    # Create short key
    df[f"{col}ShortKey"] = df[f"{col}Short"].apply(key_normalize)

In [6]:
# -- CREATE ALIAS
# Khởi tạo cột alias rỗng
for col in ['province', 'district', 'ward']:
    df[f"{col}Alias"] = np.nan

# Province alias data
province_alias_data = {
    'thanhphohanoi': ['hn'],
    'thanhphohochiminh': ['hcm'],
    'tinhbariavungtau': ['baria', 'vungtau'],
}

# District alias data (theo từng province)
district_alias_data = {
    'thanhphohochiminh': {
        'thanhphothuduc': ['quan9', 'quan2', 'q9', 'q2', 'q.9', 'q.2']
    }
}

# Ward alias data (theo từng province > district > ward)
ward_alias_data = {
    # 'thanhphohochiminh': {
    #     'thanhphothuduc': {
    #         'phuongtruongthanh': ['truongthanh', 'p.truongthanh']
    #     }
    # }
}

# Gán provinceAlias
for key, value in province_alias_data.items():
    df.loc[df['provinceKey'] == key, 'provinceAlias'] = json.dumps(value)

# Gán districtAlias
for province_key, district_data in district_alias_data.items():
    for district_key, value in district_data.items():
        df.loc[
            (df['provinceKey'] == province_key) & (df['districtKey'] == district_key),
            'districtAlias'
        ] = json.dumps(value)

# Gán wardAlias
for province_key, district_data in ward_alias_data.items():
    for district_key, ward_data in district_data.items():
        for ward_key, value in ward_data.items():
            df.loc[
                (df['provinceKey'] == province_key) &
                (df['districtKey'] == district_key) &
                (df['wardKey'] == ward_key),
                'wardAlias'
            ] = json.dumps(value)

In [7]:
# CHECK DUPLICATED FOR DISTRICT
df_district = df[['province', 'provinceKey', 'district', 'districtKey', 'districtShortKey']].drop_duplicates()

# districtKey
df_district.groupby(['province', 'districtKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False).head()
# districtKey is unique

,province,districtKey,count
0,Thành phố Cần Thơ,huyencodo,1
467,Tỉnh Quảng Nam,huyenphuninh,1
459,Tỉnh Quảng Nam,huyenbactramy,1
460,Tỉnh Quảng Nam,huyendailoc,1
461,Tỉnh Quảng Nam,huyendonggiang,1


In [8]:
# districtShortKey
count_district_short_key = df_district.groupby(['province', 'districtShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_district_short_key = count_district_short_key[count_district_short_key['count']>1].copy()
duplicated_district_short_key['districtShortKeyDuplicated'] = True
duplicated_district_short_key.drop(columns=['count'], inplace=True)
duplicated_district_short_key
# Do có Thông tin mở ngoặc nên không còn dup

,province,districtShortKey,districtShortKeyDuplicated


In [9]:
# Add flag for districtShortKey
df = pd.merge(df, duplicated_district_short_key, on=['province', 'districtShortKey'], how='left')
df['districtShortKeyDuplicated'].fillna(False, inplace=True)
df[df['districtShortKeyDuplicated']][['province', 'district', 'districtShort']].drop_duplicates().sort_values(by='districtShort')
# Dup do type, chứ không do accent, rất tốt

,province,district,districtShort


In [10]:
# CHECK DUPLICATED WARD
count_ward_key = df.groupby(['province', 'district', 'wardKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
count_ward_key['wardKeyDuplicated'] = np.where(count_ward_key['count']>1, True, False)
duplicated_ward_key = count_ward_key[count_ward_key['wardKeyDuplicated']]
duplicated_ward_key.drop(columns=['count'], inplace=True)
duplicated_ward_key

,province,district,wardKey,wardKeyDuplicated
5357,Tỉnh Nam Định,Huyện Trực Ninh,xatruchung,True
7586,Tỉnh Thanh Hóa,Huyện Hậu Lộc,xahoaloc,True
6760,Tỉnh Quảng Ngãi,Huyện Bình Sơn,xabinhthanh,True
7551,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangthanh,True
6542,Tỉnh Quảng Nam,Huyện Nam Trà My,xatradon,True
7546,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangquy,True
7545,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangphu,True
5533,Tỉnh Nghệ An,Huyện Kỳ Sơn,xanamcan,True
1295,Tỉnh An Giang,Huyện Phú Tân,xaphuthanh,True
5611,Tỉnh Nghệ An,Huyện Quỳ Châu,xachaubinh,True


In [11]:
# Add flag for wardKey
df = pd.merge(df, duplicated_ward_key, on=['province', 'district', 'wardKey'], how='left')
df['wardKeyDuplicated'].fillna(False, inplace=True)

# Change wardKey and wardShortKey to accented key
df['wardKey'] = np.where(df['wardKeyDuplicated'], df['ward'].apply(key_normalize, args=([], False)), df['wardKey'])
df['wardShortKey'] = np.where(df['wardKeyDuplicated'], df['wardShort'].apply(key_normalize, args=([], False)), df['wardShortKey'])
df[df['wardKeyDuplicated']]

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,provinceShortKey,districtKey,districtShortKey,wardKey,wardShortKey,provinceAlias,districtAlias,wardAlias,districtShortKeyDuplicated,wardKeyDuplicated
1238,Tỉnh Lào Cai,Thị xã Sa Pa,Phường Sa Pa,Lào Cai,Sa Pa,Sa Pa,Thị xã,Phường,10,22.275944,...,laocai,thixasapa,sapa,phườngsapa,sapa,NaN,NaN,NaN,False,True
1239,Tỉnh Lào Cai,Thị xã Sa Pa,Phường Sa Pả,Lào Cai,Sa Pa,Sa Pả,Thị xã,Phường,10,22.275944,...,laocai,thixasapa,sapa,phườngsapả,sapả,NaN,NaN,NaN,False,True
1807,Tỉnh Yên Bái,Huyện Trấn Yên,Xã Minh Quán,Yên Bái,Trấn Yên,Minh Quán,Huyện,Xã,15,21.683525,...,yenbai,huyentranyen,tranyen,xãminhquán,minhquán,NaN,NaN,NaN,False,True
1814,Tỉnh Yên Bái,Huyện Trấn Yên,Xã Minh Quân,Yên Bái,Trấn Yên,Minh Quân,Huyện,Xã,15,21.683525,...,yenbai,huyentranyen,tranyen,xãminhquân,minhquân,NaN,NaN,NaN,False,True
4166,Tỉnh Nam Định,Huyện Trực Ninh,Xã Trực Hưng,Nam Định,Trực Ninh,Trực Hưng,Huyện,Xã,36,20.279180,...,namdinh,huyentrucninh,trucninh,xãtrựchưng,trựchưng,NaN,NaN,NaN,False,True
4177,Tỉnh Nam Định,Huyện Trực Ninh,Xã Trực Hùng,Nam Định,Trực Ninh,Trực Hùng,Huyện,Xã,36,20.279180,...,namdinh,huyentrucninh,trucninh,xãtrựchùng,trựchùng,NaN,NaN,NaN,False,True
4579,Tỉnh Thanh Hóa,Huyện Vĩnh Lộc,Xã Vĩnh Hưng,Thanh Hóa,Vĩnh Lộc,Vĩnh Hưng,Huyện,Xã,38,19.825986,...,thanhhoa,huyenvinhloc,vinhloc,xãvĩnhhưng,vĩnhhưng,NaN,NaN,NaN,False,True
4581,Tỉnh Thanh Hóa,Huyện Vĩnh Lộc,Xã Vĩnh Hùng,Thanh Hóa,Vĩnh Lộc,Vĩnh Hùng,Huyện,Xã,38,19.825986,...,thanhhoa,huyenvinhloc,vinhloc,xãvĩnhhùng,vĩnhhùng,NaN,NaN,NaN,False,True
4716,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,Xã Hoằng Phú,Thanh Hóa,Hoằng Hóa,Hoằng Phú,Huyện,Xã,38,19.825986,...,thanhhoa,huyenhoanghoa,hoanghoa,xãhoằngphú,hoằngphú,NaN,NaN,NaN,False,True
4717,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,Xã Hoằng Quỳ,Thanh Hóa,Hoằng Hóa,Hoằng Quỳ,Huyện,Xã,38,19.825986,...,thanhhoa,huyenhoanghoa,hoanghoa,xãhoằngquỳ,hoằngquỳ,NaN,NaN,NaN,False,True


In [12]:
# wardShortKey
count_ward_short_key = df.groupby(['province', 'district', 'wardShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_short_key = count_ward_short_key[count_ward_short_key['count']>1].copy()
duplicated_ward_short_key['wardShortKeyDuplicated'] = True
duplicated_ward_short_key.drop(columns=['count'], inplace=True)
duplicated_ward_short_key

,province,district,wardShortKey,wardShortKeyDuplicated
7798,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,thieuhoa,True
4715,Tỉnh Long An,Huyện Tân Thạnh,tanthanh,True
6748,Tỉnh Quảng Ngãi,Huyện Ba Tơ,bato,True


In [13]:
# Add flag for wardShortKey
df = pd.merge(df, duplicated_ward_short_key, on=['province', 'district', 'wardShortKey'], how='left')
df['wardShortKeyDuplicated'].fillna(False, inplace=True)
df[df['wardShortKeyDuplicated']]
# Cần thêm một DICT mà wardKey là no accented nhưng wardKeyShort là accented

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,districtKey,districtShortKey,wardKey,wardShortKey,provinceAlias,districtAlias,wardAlias,districtShortKeyDuplicated,wardKeyDuplicated,wardShortKeyDuplicated
4689,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,Thị trấn Thiệu Hóa,Thanh Hóa,Thiệu Hóa,Thiệu Hóa,Huyện,Thị trấn,38,19.825986,...,huyenthieuhoa,thieuhoa,thitranthieuhoa,thieuhoa,NaN,NaN,NaN,False,False,True
4705,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,Xã Thiệu Hòa,Thanh Hóa,Thiệu Hóa,Thiệu Hòa,Huyện,Xã,38,19.825986,...,huyenthieuhoa,thieuhoa,xathieuhoa,thieuhoa,NaN,NaN,NaN,False,False,True
6360,Tỉnh Quảng Ngãi,Huyện Ba Tơ,Thị trấn Ba Tơ,Quảng Ngãi,Ba Tơ,Ba Tơ,Huyện,Thị trấn,51,15.075984,...,huyenbato,bato,thitranbato,bato,NaN,NaN,NaN,False,False,True
6373,Tỉnh Quảng Ngãi,Huyện Ba Tơ,Xã Ba Tô,Quảng Ngãi,Ba Tơ,Ba Tô,Huyện,Xã,51,15.075984,...,huyenbato,bato,xabato,bato,NaN,NaN,NaN,False,False,True
8520,Tỉnh Long An,Huyện Tân Thạnh,Thị trấn Tân Thạnh,Long An,Tân Thạnh,Tân Thạnh,Huyện,Thị trấn,80,10.685145,...,huyentanthanh,tanthanh,thitrantanthanh,tanthanh,NaN,NaN,NaN,False,False,True
8528,Tỉnh Long An,Huyện Tân Thạnh,Xã Tân Thành,Long An,Tân Thạnh,Tân Thành,Huyện,Xã,80,10.685145,...,huyentanthanh,tanthanh,xatanthanh,tanthanh,NaN,NaN,NaN,False,False,True


In [14]:
# Create keywords
for col in unit_cols:
    level = level_map[col]
    df[f"{col}Keywords"] = df.apply(lambda row: create_keywords(row, level=level), axis=1)

In [15]:
# Province map
df_province = df[['provinceKey', 'provinceKeywords', 'province', 'provinceShort', 'provinceLat', 'provinceLon']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE = {}
for _, row in df_province.iterrows():
    DICT_PROVINCE[row['provinceKey']] = {
        'provinceKeywords': json.loads(row['provinceKeywords']),
        'province': row['province'],
        'provinceShort': row['provinceShort'],
        'provinceLat': row['provinceLat'],
        'provinceLon': row['provinceLon'],
    }


# District map
df_district = df[['provinceKey', 'provinceShortKey', 'districtKey', 'districtShortKey', 'districtKeywords', 'district', 'districtType', 'districtShort', 'districtLat', 'districtLon']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE_DISTRICT = {}
for _, province_row in df_province.iterrows():
    province_key = province_row['provinceKey']
    DICT_PROVINCE_DISTRICT[province_key] = {}

    df_district_filtered = df_district[df_district['provinceKey'] == province_key]

    for _, district_row in df_district_filtered.iterrows():
        DICT_PROVINCE_DISTRICT[province_key][district_row['districtKey']] = {
            'districtKeywords': json.loads(district_row['districtKeywords']) if pd.notnull(district_row['districtKeywords']) else [],
            'district': district_row['district'],
            'districtType': district_row['districtType'],
            'districtShort': district_row['districtShort'],
            'districtLat': district_row['districtLat'],
            'districtLon': district_row['districtLon'],
        }


# Unique district to province map
province_short_keys = df['provinceShortKey'].unique().tolist()
for index, row in df_district.iterrows():
    district_short_key = row['districtShortKey']
    left_district_short_keys = df_district.loc[df_district.index != index, 'districtShortKey'].tolist()
    if district_short_key not in province_short_keys and district_short_key not in left_district_short_keys:
        df_district.loc[index, 'districtUnique'] = True
df_district['districtUnique'].fillna(False, inplace=True)
df_district_unique = df_district[df_district['districtUnique']==True]

DICT_UNIQUE_DISTRICT_PROVINCE = {}
for _, row in df_district_unique.iterrows():
    DICT_UNIQUE_DISTRICT_PROVINCE[row['districtKey']] = {
        'districtKeywords': json.loads(row['districtKeywords']),
        'provinceKey': row['provinceKey']
    }

In [16]:
# Ward map
df_ward = df[['provinceKey', 'districtKey', 'wardKey', 'wardKeywords', 'ward', 'wardShort', 'wardType', 'wardKeyDuplicated', 'wardLat', 'wardLon']].drop_duplicates().reset_index(drop=True)

df_ward_no_accented = df_ward[df_ward['wardKeyDuplicated']==False]
df_ward_accented = df_ward[df_ward['wardKeyDuplicated']==True]

def build_province_district_ward_dict(df, short_name_key=False):
    DICT_PROVINCE_DISTRICT_WARD = {}

    for province_key, province_group in df.groupby('provinceKey'):
        DICT_PROVINCE_DISTRICT_WARD[province_key] = {}

        for district_key, district_group in province_group.groupby('districtKey'):
            DICT_PROVINCE_DISTRICT_WARD[province_key][district_key] = {}

            for _, row in district_group.iterrows():
                ward_key = row['wardKey']
                if short_name_key:
                    keywords = [key_normalize(row['wardShort'], decode=False)]
                else:
                    keywords = json.loads(row['wardKeywords']) if pd.notnull(row['wardKeywords']) else []
                DICT_PROVINCE_DISTRICT_WARD[province_key][district_key][ward_key] = {
                    'wardKeywords': keywords,
                    'ward': row['ward'],
                    'wardShort': row['wardShort'],
                    'wardType': row['wardType'],
                    'wardLat': row['wardLat'],
                    'wardLon': row['wardLon'],
                }

    return DICT_PROVINCE_DISTRICT_WARD


DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED = build_province_district_ward_dict(df_ward_no_accented)
DICT_PROVINCE_DISTRICT_WARD_ACCENTED = build_province_district_ward_dict(df_ward_accented)

df_ward_short_accented = df[df['wardShortKeyDuplicated']]
DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED = build_province_district_ward_dict(df_ward_short_accented, short_name_key=True)

In [17]:
DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED

{'tinhlongan': {'huyentanthanh': {'thitrantanthanh': {'wardKeywords': ['tânthạnh'],
    'ward': 'Thị trấn Tân Thạnh',
    'wardShort': 'Tân Thạnh',
    'wardType': 'Thị trấn',
    'wardLat': 10.603397,
    'wardLon': 106.049323},
   'xatanthanh': {'wardKeywords': ['tânthành'],
    'ward': 'Xã Tân Thành',
    'wardShort': 'Tân Thành',
    'wardType': 'Xã',
    'wardLat': 10.575417,
    'wardLon': 105.905369}}},
 'tinhquangngai': {'huyenbato': {'thitranbato': {'wardKeywords': ['batơ'],
    'ward': 'Thị trấn Ba Tơ',
    'wardShort': 'Ba Tơ',
    'wardType': 'Thị trấn',
    'wardLat': 14.763479,
    'wardLon': 108.733367},
   'xabato': {'wardKeywords': ['batô'],
    'ward': 'Xã Ba Tô',
    'wardShort': 'Ba Tô',
    'wardType': 'Xã',
    'wardLat': 14.713395,
    'wardLon': 108.629438}}},
 'tinhthanhhoa': {'huyenthieuhoa': {'thitranthieuhoa': {'wardKeywords': ['thiệuhóa'],
    'ward': 'Thị trấn Thiệu Hóa',
    'wardShort': 'Thiệu Hóa',
    'wardType': 'Thị trấn',
    'wardLat': 19.8865,
   

In [18]:
parser_data = {
    'DICT_PROVINCE': DICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT': DICT_PROVINCE_DISTRICT,
    'DICT_UNIQUE_DISTRICT_PROVINCE': DICT_UNIQUE_DISTRICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED,
    'DICT_PROVINCE_DISTRICT_WARD_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_ACCENTED,
    'DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED
}

In [19]:
df[df['districtShortDuplicated']==True]

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,wardShortKey,provinceAlias,districtAlias,wardAlias,districtShortKeyDuplicated,wardKeyDuplicated,wardShortKeyDuplicated,provinceKeywords,districtKeywords,wardKeywords
5499,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Xuân,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Xuân,Huyện,Xã,42,18.245968,...,kyxuan,NaN,NaN,NaN,False,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakyxuan"", ""kyxuan""]"
5500,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Bắc,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Bắc,Huyện,Xã,42,18.245968,...,kybac,NaN,NaN,NaN,False,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakybac"", ""kybac""]"
5501,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Phú,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Phú,Huyện,Xã,42,18.245968,...,kyphu,NaN,NaN,NaN,False,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakyphu"", ""kyphu""]"
5502,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Phong,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Phong,Huyện,Xã,42,18.245968,...,kyphong,NaN,NaN,NaN,False,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakyphong"", ""kyphong""]"
5503,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Tiến,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Tiến,Huyện,Xã,42,18.245968,...,kytien,NaN,NaN,NaN,False,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakytien"", ""kytien""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9763,Tỉnh Hậu Giang,Thị xã Long Mỹ,Phường Vĩnh Tường,Hậu Giang,Long Mỹ (Thị xã),Vĩnh Tường,Thị xã,Phường,93,9.757898,...,vinhtuong,NaN,NaN,NaN,False,False,False,"[""tinhhaugiang"", ""haugiang""]","[""thixalongmy"", ""longmythixa"", ""txlongmy""]","[""phuongvinhtuong"", ""vinhtuong""]"
9764,Tỉnh Hậu Giang,Thị xã Long Mỹ,Xã Long Trị,Hậu Giang,Long Mỹ (Thị xã),Long Trị,Thị xã,Xã,93,9.757898,...,longtri,NaN,NaN,NaN,False,False,False,"[""tinhhaugiang"", ""haugiang""]","[""thixalongmy"", ""longmythixa"", ""txlongmy""]","[""xalongtri"", ""longtri""]"
9765,Tỉnh Hậu Giang,Thị xã Long Mỹ,Xã Long Trị A,Hậu Giang,Long Mỹ (Thị xã),Long Trị A,Thị xã,Xã,93,9.757898,...,longtria,NaN,NaN,NaN,False,False,False,"[""tinhhaugiang"", ""haugiang""]","[""thixalongmy"", ""longmythixa"", ""txlongmy""]","[""xalongtria"", ""longtria""]"
9766,Tỉnh Hậu Giang,Thị xã Long Mỹ,Xã Long Phú,Hậu Giang,Long Mỹ (Thị xã),Long Phú,Thị xã,Xã,93,9.757898,...,longphu,NaN,NaN,NaN,False,False,False,"[""tinhhaugiang"", ""haugiang""]","[""thixalongmy"", ""longmythixa"", ""txlongmy""]","[""xalongphu"", ""longphu""]"


In [20]:
df[df['districtShortKeyDuplicated']==True]

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,wardShortKey,provinceAlias,districtAlias,wardAlias,districtShortKeyDuplicated,wardKeyDuplicated,wardShortKeyDuplicated,provinceKeywords,districtKeywords,wardKeywords


In [21]:
df[df['wardShortDuplicated']==True]

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,wardShortKey,provinceAlias,districtAlias,wardAlias,districtShortKeyDuplicated,wardKeyDuplicated,wardShortKeyDuplicated,provinceKeywords,districtKeywords,wardKeywords
165,Thành phố Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Hà Nội,Gia Lâm,Yên Viên (Thị trấn),Huyện,Thị trấn,1,21.027764,...,yenvienthitran,"[""hn""]",NaN,NaN,False,False,False,"[""thanhphohanoi"", ""hanoi"", ""hn""]","[""huyengialam"", ""gialam""]","[""yenvienthitran"", ""thitranyenvien"", ""ttyenvien""]"
167,Thành phố Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Hà Nội,Gia Lâm,Yên Viên (Xã),Huyện,Xã,1,21.027764,...,yenvienxa,"[""hn""]",NaN,NaN,False,False,False,"[""thanhphohanoi"", ""hanoi"", ""hn""]","[""huyengialam"", ""gialam""]","[""xayenvien"", ""yenvienxa"", ""xyenvien""]"
1425,Tỉnh Lai Châu,Huyện Mường Tè,Thị trấn Mường Tè,Lai Châu,Mường Tè,Mường Tè (Thị trấn),Huyện,Thị trấn,12,22.355483,...,muongtethitran,NaN,NaN,NaN,False,False,False,"[""tinhlaichau"", ""laichau""]","[""huyenmuongte"", ""muongte""]","[""thitranmuongte"", ""muongtethitran"", ""ttmuongte""]"
1430,Tỉnh Lai Châu,Huyện Mường Tè,Xã Mường Tè,Lai Châu,Mường Tè,Mường Tè (Xã),Huyện,Xã,12,22.355483,...,muongtexa,NaN,NaN,NaN,False,False,False,"[""tinhlaichau"", ""laichau""]","[""huyenmuongte"", ""muongte""]","[""muongtexa"", ""xamuongte"", ""xmuongte""]"
1648,Tỉnh Sơn La,Huyện Mai Sơn,Thị trấn Hát Lót,Sơn La,Mai Sơn,Hát Lót (Thị trấn),Huyện,Thị trấn,14,21.172464,...,hatlotthitran,NaN,NaN,NaN,False,False,False,"[""tinhsonla"", ""sonla""]","[""huyenmaison"", ""maison""]","[""thitranhatlot"", ""hatlotthitran"", ""tthatlot""]"
1658,Tỉnh Sơn La,Huyện Mai Sơn,Xã Hát Lót,Sơn La,Mai Sơn,Hát Lót (Xã),Huyện,Xã,14,21.172464,...,hatlotxa,NaN,NaN,NaN,False,False,False,"[""tinhsonla"", ""sonla""]","[""huyenmaison"", ""maison""]","[""xahatlot"", ""hatlotxa"", ""xhatlot""]"
1820,Tỉnh Yên Bái,Huyện Trạm Tấu,Thị trấn Trạm Tấu,Yên Bái,Trạm Tấu,Trạm Tấu (Thị trấn),Huyện,Thị trấn,15,21.683525,...,tramtauthitran,NaN,NaN,NaN,False,False,False,"[""tinhyenbai"", ""yenbai""]","[""huyentramtau"", ""tramtau""]","[""tramtauthitran"", ""thitrantramtau"", ""tttramtau""]"
1825,Tỉnh Yên Bái,Huyện Trạm Tấu,Xã Trạm Tấu,Yên Bái,Trạm Tấu,Trạm Tấu (Xã),Huyện,Xã,15,21.683525,...,tramtauxa,NaN,NaN,NaN,False,False,False,"[""tinhyenbai"", ""yenbai""]","[""huyentramtau"", ""tramtau""]","[""xatramtau"", ""tramtauxa"", ""xtramtau""]"
2345,Tỉnh Lạng Sơn,Huyện Chi Lăng,Thị trấn Chi Lăng,Lạng Sơn,Chi Lăng,Chi Lăng (Thị trấn),Huyện,Thị trấn,20,21.856371,...,chilangthitran,NaN,NaN,NaN,False,False,False,"[""tinhlangson"", ""langson""]","[""huyenchilang"", ""chilang""]","[""chilangthitran"", ""thitranchilang"", ""ttchilang""]"
2363,Tỉnh Lạng Sơn,Huyện Chi Lăng,Xã Chi Lăng,Lạng Sơn,Chi Lăng,Chi Lăng (Xã),Huyện,Xã,20,21.856371,...,chilangxa,NaN,NaN,NaN,False,False,False,"[""tinhlangson"", ""langson""]","[""huyenchilang"", ""chilang""]","[""chilangxa"", ""xachilang"", ""xchilang""]"


In [22]:
df[df['wardShortKeyDuplicated']==True]

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,wardShortKey,provinceAlias,districtAlias,wardAlias,districtShortKeyDuplicated,wardKeyDuplicated,wardShortKeyDuplicated,provinceKeywords,districtKeywords,wardKeywords
4689,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,Thị trấn Thiệu Hóa,Thanh Hóa,Thiệu Hóa,Thiệu Hóa,Huyện,Thị trấn,38,19.825986,...,thieuhoa,NaN,NaN,NaN,False,False,True,"[""tinhthanhhoa"", ""thanhhoa""]","[""huyenthieuhoa"", ""thieuhoa""]","[""thieuhoathitran"", ""thitranthieuhoa"", ""ttthie..."
4705,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,Xã Thiệu Hòa,Thanh Hóa,Thiệu Hóa,Thiệu Hòa,Huyện,Xã,38,19.825986,...,thieuhoa,NaN,NaN,NaN,False,False,True,"[""tinhthanhhoa"", ""thanhhoa""]","[""huyenthieuhoa"", ""thieuhoa""]","[""thieuhoaxa"", ""xathieuhoa"", ""xthieuhoa""]"
6360,Tỉnh Quảng Ngãi,Huyện Ba Tơ,Thị trấn Ba Tơ,Quảng Ngãi,Ba Tơ,Ba Tơ,Huyện,Thị trấn,51,15.075984,...,bato,NaN,NaN,NaN,False,False,True,"[""tinhquangngai"", ""quangngai""]","[""huyenbato"", ""bato""]","[""thitranbato"", ""batothitran"", ""ttbato""]"
6373,Tỉnh Quảng Ngãi,Huyện Ba Tơ,Xã Ba Tô,Quảng Ngãi,Ba Tơ,Ba Tô,Huyện,Xã,51,15.075984,...,bato,NaN,NaN,NaN,False,False,True,"[""tinhquangngai"", ""quangngai""]","[""huyenbato"", ""bato""]","[""batoxa"", ""xabato"", ""xbato""]"
8520,Tỉnh Long An,Huyện Tân Thạnh,Thị trấn Tân Thạnh,Long An,Tân Thạnh,Tân Thạnh,Huyện,Thị trấn,80,10.685145,...,tanthanh,NaN,NaN,NaN,False,False,True,"[""tinhlongan"", ""longan""]","[""huyentanthanh"", ""tanthanh""]","[""thitrantanthanh"", ""tanthanhthitran"", ""tttant..."
8528,Tỉnh Long An,Huyện Tân Thạnh,Xã Tân Thành,Long An,Tân Thạnh,Tân Thành,Huyện,Xã,80,10.685145,...,tanthanh,NaN,NaN,NaN,False,False,True,"[""tinhlongan"", ""longan""]","[""huyentanthanh"", ""tanthanh""]","[""tanthanhxa"", ""xatanthanh"", ""xtanthanh""]"


In [23]:
with open(BASE_DIR / 'vietnamadminunits/data/parser_63.json', 'w') as f:
    json.dump(parser_data, f)

In [24]:
import sqlite3
with sqlite3.connect(BASE_DIR / 'vietnamadminunits/data/dataset.db') as conn:
    df.to_sql('admin_units_63', conn, if_exists='replace', index=False)

In [25]:
df.to_csv(BASE_DIR / 'data/interim/63_provinces_10040_wards_with_location_and_key.csv', index=False)